## **| 분류 연습 문제**
___
출처 : 핸즈온 머신러닝 Ch03 분류 연습문제 1, 2번

In [1]:
# import data
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1, as_frame = False)

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
X, y = mnist["data"], mnist["target"]

In [3]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### **1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.**
___

1. `KNeghtborsClassifier`를 사용하는 것을 추천합니다.
2. `weights`와 `n_neighbors` 하이퍼 파라미터로 그리드 탐색을 시도하여, 좋은 하이퍼 파라미터 값을 찾아보세요.

In [5]:
# import package

from sklearn.neighbors import KNeighborsClassifier


knn_clf = KNeighborsClassifier()

In [8]:
# Try GridSearch to optimize hyperparameter

from sklearn.model_selection import GridSearchCV
import numpy as np


param_grid = {'n_neighbors':np.arange(1, 6)} 

grid = GridSearchCV(knn_clf, param_grid, cv = 5, scoring = 'accuracy')

In [9]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([1, 2, 3, 4, 5])},
             scoring='accuracy')

In [10]:
# best hyperparameter


print(grid.best_score_)



0.9700666666666666


In [11]:
# best score

print(grid.best_params_)



{'n_neighbors': 3}


In [12]:
# model test


best_model = grid.best_estimator_


# 테스트 데이터로 모델 평가
test_accuracy = best_model.score(X_test, y_test)
print("테스트 데이터의 정확도:", test_accuracy)


테스트 데이터의 정확도: 0.9705


### **2. 다음 단계를 따라 인위적으로 훈련 세트를 늘리는 데이터 증식 또는 훈련 세트 확장 기법을 연습해봅시다.**
___

#### **STEP 1. MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어 보세요.**

In [13]:
import numpy as np

def shift_image(image, direction):

    shifted_image = np.copy(image)
    
    # 이동 방향에 따라 픽셀 이동
    if direction == 'left':
        shifted_image[:, :-1] = shifted_image[:, 1:]
        shifted_image[:, -1] = 0  
    elif direction == 'right':
        shifted_image[:, 1:] = shifted_image[:, :-1]
        shifted_image[:, 0] = 0 
    elif direction == 'up':
        shifted_image[:-1, :] = shifted_image[1:, :]
        shifted_image[-1, :] = 0 
    elif direction == 'down':
        shifted_image[1:, :] = shifted_image[:-1, :]
        shifted_image[0, :] = 0 
    
    return shifted_image


####  **STEP 2. 앞에서 만든 함수를 이용하여, 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요**

In [15]:
left_image = shift_image(X_train, "left")

right_image = shift_image(X_train, "right")

up_image = shift_image(X_train, "up")

down_image = shift_image(X_train, "down")

In [17]:

combined_X_train = np.concatenate((X_train, left_image, right_image, up_image, down_image), axis=0)


combined_y_train = np.concatenate((y_train, y_train, y_train, y_train, y_train), axis=0)


> 모범답안 

In [30]:
from scipy.ndimage import shift

def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])



In [31]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

In [32]:
for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)

X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

####  **STEP 3. 위에서 확장한 데이터셋을 이용하여, 1번 문제에서 찾은 최적 모델을 훈련시키고, 테스트 세트에서 정확도를 측정해보세요**

In [18]:
knn_clf = KNeighborsClassifier(n_neighbors=3)

In [19]:
knn_clf.fit(combined_X_train, combined_y_train)

KNeighborsClassifier(n_neighbors=3)

In [20]:
from sklearn.metrics import accuracy_score

y_pred = knn_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(accuracy)


0.5958


In [33]:
knn_clf.fit(X_train_augmented, y_train_augmented)

KNeighborsClassifier(n_neighbors=3)

In [34]:
from sklearn.metrics import accuracy_score

y_pred = knn_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(accuracy)


0.9756
